In [1]:
from time import sleep
import requests
import numpy as np
import pandas as pd
import json

In [2]:
url = "http://live.pseresults.com/e/83/s/1?last_name=&first_name=a&city=&bib="
url = "http://live.pseresults.com/e/83/s/1?first_name=a"
response = requests.get(url, timeout=5)
json.loads(response.content)

{u'current_page': 1,
 u'database_columns': [u'place',
  u'name',
  u'residence',
  u'bib',
  u'chip_time',
  u'pace',
  u'category',
  u'category_place',
  u'gender_place',
  u'split1',
  u'split2',
  u'split3',
  u'split4',
  u'split5',
  u'split6',
  u'split8'],
 u'display_columns': [u'Place',
  u'Name',
  u'Residence',
  u'Bib #',
  u'Time',
  u'Pace',
  u'Category',
  u'Category Place',
  u'Gender Place',
  u'5KM',
  u'10KM',
  u'15KM',
  u'21KM',
  u'25KM',
  u'30KM',
  u'40KM'],
 u'per_page': 25,
 u'timing_rows': [[4521,
   u'Alan Abad',
   u'Mililani',
   u'1002',
   u'4:57:50',
   u'11:22',
   u'M35-39',
   u'379/1121',
   u'3097/10838',
   u'24:36',
   u'49:07',
   u'1:16:51',
   u'1:53:21',
   u'2:27:04',
   u'3:09:25',
   u'4:39:17',
   u'1002'],
  [7693,
   u'Alana Abbey',
   u'Kaneohe',
   u'6207',
   u'5:40:37',
   u'13:00',
   u'W25-29',
   u'319/1045',
   u'2790/9512',
   u'29:44',
   u'1:01:13',
   u'1:39:04',
   u'2:26:27',
   u'2:59:04',
   u'3:43:54',
   u'',
   u'6

In [3]:
# 2017 Honolulu

# URL format:
# http://live.pseresults.com/e/83/s/1?last_name=&first_name=a&city=&bib=
# http://live.pseresults.com/e/83/s/1?first_name=a
begin_url = "http://live.pseresults.com/e/83/s/"
end_url = "?first_name="

alphabet = "abcdefghijklmnopqrstuvwxyz"
counter = 0
total_records = 0

for x in alphabet[0:]:
    response = requests.get(begin_url + '1' + end_url + x, timeout=5)
    num_pages = json.loads(response.content)['total_pages']
    num_rows = json.loads(response.content)['total_rows']
    total_records += num_rows
    
    print("{} {} {}".format(x, num_pages, num_rows))
    
    for y in range(1,num_pages+1):
        response = requests.get(begin_url + str(y) + end_url + x, timeout=5)
        filename = "testing_" + str(counter) + ".xlsx"
        pd.DataFrame(pd.DataFrame(json.loads(response.content)['timing_rows'])).to_excel(filename, index=None)
        counter += 1
        sleep(0.5) # be nice
        
print("Total records: {}".format(total_records))

a 73 1808
b 23 568
c 49 1214
d 37 903
e 31 769
f 11 273
g 17 408
h 59 1464
i 13 314
j 75 1865
k 113 2818
l 26 632
m 137 3425
n 43 1071
o 5 103
p 15 367
q 1 13
r 55 1373
s 90 2237
t 91 2258
u 2 28
v 5 119
w 11 261
x 2 26
y 80 1984
z 2 45
Total records: 26346


In [4]:
temp = pd.DataFrame()
for x in range(0,2000):
    filename = "testing_" + str(x) + ".xlsx"
    try:
        temp = pd.concat([temp, pd.read_excel(filename)])
    except:
        print("stopped at {}!!".format(filename))
        break
print(len(temp))

temp.columns = ['place','name','city','bib','chip','pace','AG','AG_place','sex_place','split1_time','split2_time','split3_time','split4_time','split5_time','split6_time','split7_time','bib2']
temp = temp.drop_duplicates()
del temp['bib2']
print(len(temp))

temp['AG'].replace(np.nan,"",inplace=True)
temp['age'] = temp.apply(lambda row: 30 if "Elite" in row['AG']
                         else 0 if "No Age" in row['AG']
                         else 0 if len(row['AG']) == 0
                         else row['AG'][-2:], axis=1)
temp['sex'] = temp.apply(lambda row: "F" if "W" in row['AG']
                         else "M", axis=1)
temp.age = temp.age.apply(int)
temp.bib = temp.bib.apply(str)

temp.sort_values(by=['place','bib']).to_excel("testing.xlsx",index=None)
# 1066 files, 26346 records (first name)

stopped at testing_1066.xlsx!!
26346
26346
